<a href="https://colab.research.google.com/github/lukmanfauzirosidin/lukmanfauzirosidin.github.io/blob/master/kajojo_analyst.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import auth
auth.authenticate_user()

import gspread
from google.auth import default
creds, _ = default()
import numpy as np
gc = gspread.authorize(creds)

worksheet = gc.open('Database Kajojo').sheet1

# get_all_values gives a list of rows.
rows = worksheet.get_all_values()

# Convert to a DataFrame and render.
import pandas as pd
df = pd.DataFrame.from_records(rows[1:], columns=rows[0])

In [2]:
df.head(5)

,No Faktur,Tanggal,Nama Outlet,Nama Kasir,Jam,Kategori,Deskripsi Produk,Produk,Ice or Hot,Modifier,...,Tipe Diskon per Barang,Diskon Transaksi,Tipe Diskon Transaksi,Pajak,Subtotal,Status,Metode Pembayaran,Tipe,,
0,16113C6M,10/06/2021,Pusat,Kajojo,13:23:35,Coffee,"Americano, Ice",Americano,Ice,,...,,0,,0,Rp0,Dibatalkan,CASH,Offline,,
1,16113TV9,10/06/2021,Pusat,Kajojo,13:22:56,Coffee,"Americano, Ice",Americano,Ice,,...,,0,,0,Rp0,Dibatalkan,CASH,Offline,GoFood,Online
2,16113O0L,10/06/2021,Pusat,Kajojo,13:21:24,Coffee,"Americano, Ice",Americano,Ice,,...,,0,,0,Rp0,Dibatalkan,CASH,Offline,GrabFood,Online
3,16113OKG,10/06/2021,Pusat,Kajojo,13:20:51,Coffee,"Americano, Ice",Americano,Ice,,...,,0,,0,Rp0,Dibatalkan,CASH,Offline,ShopFood,Online
4,16822N6C,17/06/2021,Pusat,Rizal,22:49:42,Coffee,"Coffee Latte, Ice",Coffee Latte,Ice,,...,,0,,0,Rp0,Dibatalkan,CASH,Offline,,


Add Month

In [3]:
df['Month'] = df['Tanggal'].str[3:5]
df['Month'] = df['Month'].astype(int)
df = df[df['Status'] != "Dibatalkan"]

In [4]:
df['Years'] = df['Tanggal'].str[6:]
df['Years'] = df['Years'].astype(int)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [5]:
df['Subtotal'] = df['Subtotal'].replace({'Rp':'', '\.':''}, regex = True)
df['Subtotal'] = df['Subtotal'].astype(int)


Convert Dtypes

In [6]:
#df['Barang Terjual'] = df['Barang Terjual'].astype(int)
df['Jumlah Barang'] = df['Jumlah Barang'].astype(int)
df['Jumlah Harga'] = df['Jumlah Harga'].astype(int)
df['Tipe'] = df['Tipe'].astype(object)

group produk by no faktur

In [7]:
df['Grouped'] = df.groupby('No Faktur')['Produk '].transform(lambda x: ','.join(x))
dft = df[['No Faktur', 'Grouped']].drop_duplicates()

In [8]:
df

,No Faktur,Tanggal,Nama Outlet,Nama Kasir,Jam,Kategori,Deskripsi Produk,Produk,Ice or Hot,Modifier,...,Pajak,Subtotal,Status,Metode Pembayaran,Tipe,,,Month,Years,Grouped
8,16821G6X,17/06/2021,Pusat,Rizal,21:03:40,Coffee,"Avocado Latte, Ice",Avocado Latte,Ice,,...,0,17000,Transaksi,QRIS,Offline,,,6,2021,"Avocado Latte,Hazelnut Latte,Regal Kajojo,sofood"
9,16821QYG,17/06/2021,Pusat,Rizal,21:01:01,Coffee,"Coffee Latte, Ice",Coffee Latte,Ice,,...,0,15000,Transaksi,QRIS,Offline,,,6,2021,"Coffee Latte,Regal Kajojo,sofood"
10,16821G6X,17/06/2021,Pusat,Rizal,21:03:40,Coffee,"Hazelnut Latte, Ice",Hazelnut Latte,Ice,,...,0,17000,Transaksi,QRIS,Offline,,,6,2021,"Avocado Latte,Hazelnut Latte,Regal Kajojo,sofood"
11,16821G6X,17/06/2021,Pusat,Rizal,21:03:40,Cookies,"Regal Kajojo,",Regal Kajojo,,,...,0,15000,Transaksi,QRIS,Offline,,,6,2021,"Avocado Latte,Hazelnut Latte,Regal Kajojo,sofood"
12,16821QYG,17/06/2021,Pusat,Rizal,21:01:01,Cookies,"Regal Kajojo,",Regal Kajojo,,,...,0,15000,Transaksi,QRIS,Offline,,,6,2021,"Coffee Latte,Regal Kajojo,sofood"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16974,19320Z4I,11/07/2020,Pusat,Kajojo,20:00:37,#N/A,"Soerabi Meses,",Soerabi Meses,,,...,0,7000,Transaksi,CASH,Offline,,,7,2020,"Coffee Latte,Cookies & Cream Oreo,Regal,Soerab..."
16975,19320Z4I,11/07/2020,Pusat,Kajojo,20:00:37,#N/A,"Soerabi Tiramisu,",Soerabi Tiramisu,,,...,0,7000,Transaksi,CASH,Offline,,,7,2020,"Coffee Latte,Cookies & Cream Oreo,Regal,Soerab..."
16976,19317WG0,11/07/2020,Pusat,Kajojo,17:20:09,#N/A,"Coffee Latte, Caramel Latte",Coffee Latte,Caramel Latte,,...,0,14000,Transaksi,CASH,Offline,,,7,2020,"Coffee Latte,Regal"
16977,19317WG0,11/07/2020,Pusat,Kajojo,17:20:09,#N/A,"Regal,",Regal,,,...,0,12000,Transaksi,CASH,Offline,,,7,2020,"Coffee Latte,Regal"


In [9]:
dft

,No Faktur,Grouped
8,16821G6X,"Avocado Latte,Hazelnut Latte,Regal Kajojo,sofood"
9,16821QYG,"Coffee Latte,Regal Kajojo,sofood"
15,169208RP,"Coffee Latte,Susu Caramel Latte,Susu Caramel L..."
16,16915OQ6,Coffee Latte
17,16917RST,Hazelnut Latte
...,...,...
16966,193219WR,"Coffee Latte,Matcha,Regal"
16969,19320HHA,"Cookies & Cream Oreo,Matcha"
16971,19320Z4I,"Coffee Latte,Cookies & Cream Oreo,Regal,Soerab..."
16976,19317WG0,"Coffee Latte,Regal"


Pasangan produk berdasarkan transaksi

In [10]:
from itertools import combinations
from collections import Counter

count = Counter()

for row in dft['Grouped']:
	row_list = row.split(',')
	count.update(Counter(combinations(row_list, 2)))

for key, value in count.most_common(20):
	print(key, value)

('Coffee Latte', 'Ice Cream Croffle') 149
('Coffee Latte', 'Coffee Latte') 132
('Coffee Latte', 'Kentang Goreng') 127
('Coffee Latte', 'Cuanki Ngeunah') 97
('Ice Cream Croffle', 'Kentang Goreng') 95
('Coffee Latte', 'Cookies & Cream Oreo') 82
('Ice Cream Croffle', 'Lotus Croffle') 79
('Caramel Latte', 'Coffee Latte') 74
('Ice Cream Croffle', 'Tiramisu Croffle') 71
('Ice Cream Croffle', 'Cuanki Ngeunah') 71
('Aren Latte', 'Coffee Latte') 70
('Coffee Latte', 'Hazelnut Latte') 69
('Coffee Latte', 'Matcha') 59
('Coffee Latte', 'Rotbar Cokelat Crunchy') 59
('Choco Crunch Croffle', 'Ice Cream Croffle') 58
('Coffee Latte', 'Regal') 57
('Coffee Latte', 'Regal Kajojo') 52
('Ice Cream Croffle', 'Meekajo') 52
('Hazelnut Latte', 'Kentang Goreng') 47
('Coffee Latte', 'Meekajo') 47


In [11]:
df.describe()

,Jumlah Barang,Jumlah Harga,Subtotal,Month,Years
count,16133.000000,16133.000000,16133.000000,16133.000000,16133.000000
mean,1.109961,15931.227546,15605.468853,6.835554,2021.027645
std,0.408116,6555.905134,6575.335286,4.035476,0.671572
min,1.000000,0.000000,0.000000,1.000000,2020.000000
25%,1.000000,12000.000000,12000.000000,3.000000,2021.000000
50%,1.000000,15000.000000,15000.000000,8.000000,2021.000000
75%,1.000000,18000.000000,17250.000000,11.000000,2021.000000
max,10.000000,135000.000000,135000.000000,12.000000,2022.000000


In [12]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 16133 entries, 8 to 16978
Data columns (total 28 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   No Faktur               16133 non-null  object
 1   Tanggal                 16133 non-null  object
 2   Nama Outlet             16133 non-null  object
 3   Nama Kasir              16133 non-null  object
 4   Jam                     16133 non-null  object
 5   Kategori                16133 non-null  object
 6   Deskripsi Produk        16133 non-null  object
 7   Produk                  16133 non-null  object
 8   Ice or Hot              16133 non-null  object
 9   Modifier                16133 non-null  object
 10  Jumlah Barang           16133 non-null  int64 
 11  Jumlah dibatalkan       16133 non-null  object
 12  Harga Per Barang        16133 non-null  object
 13  Jumlah Harga            16133 non-null  int64 
 14  Diskon Per Barang       16133 non-null  object
 15  Ti

In [13]:
df['Barang Terjual'] = df['Barang Terjual'].astype(int)

KeyError: ignored

In [ ]:
df['Diskon Transaksi'] = df['Diskon Transaksi'].replace({',':'.'}, regex = True)

In [ ]:
df['Diskon Per Barang'] = df['Diskon Per Barang'].astype(float)

In [ ]:
df.info()

In [ ]:
df.groupby(['Month', 'Years']).describe()

In [ ]:
df.isna().sum()